In [ ]:
#reload edited modules automatically
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'plaidml.keras.backend'
from utils import create_disjoint_cifar10_task,Cifar10
from model import cnn_model
import matplotlib.pyplot as plt
from information import Info
from copy import deepcopy


In [ ]:
#igore the warning messages. Cause the kal drawback is slower than normal process, keras 
#will print some warning messages.
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
#Plot the accuracy of test data
#Parameters:
# - name: the name of the model. It will be used in label
# - acc: list of accuracy
# - data_num: which data is plotted(D1,D2 or D3)
def acc_plot(name,acc,data_num):
	plt.figure(1)
	sub = '31'+str(data_num)
	plt.subplot(sub)
	plt.title('test accuracy on {}th dataset'.format(data_num))
	plt.plot(acc,label=name)
	plt.ylabel('acc')
	plt.xlabel('training time')
	for i in range(len(acc)-1):
		plt.vlines((i+1),0,1,color='r',linestyles='dashed')
	plt.legend(loc='upper right')
	plt.subplots_adjust(wspace=1,hspace=1)
	plt.savefig('./images/permuted.png'.format(name))
    display.display(plt.gcf())
    display.clear_output(wait=True)
    
def plot_history(file_name,model,shift=10):
    plt.subplot(211)
    plt.title('accuracy on current training data')
    for i in range(shift):
        plt.vlines(self.epoch*(i+1),0,1,color='r',linestyles='dashed')

    plt.plot(self.history.history['acc'],label='{}'.format(model))
    plt.ylabel('acc')
    plt.xlabel('training time')
    plt.legend(loc='upper right')
    plt.subplot(212)
    plt.title('validation accuracy on original data')
    plt.plot(self.history.history['val_acc'],label='{}'.format(model))
    plt.ylabel('acc')
    plt.xlabel('training time')
    for i in range(shift):
        plt.vlines(self.epoch*(i+1),0,1,color='r',linestyles='dashed')
    plt.legend(loc='upper right')
    plt.subplots_adjust(wspace=1,hspace=1)
    plt.savefig('./images/{}.png'.format(file_name))
    display.display(plt.gcf())
    display.clear_output(wait=True)

In [ ]:
def test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3):
    acc_test_d1.append(model.evaluate(task.test.images[test_index[0]],task.test.labels[test_index[0]]))
    acc_test_d2.append(model.evaluate(task.test.images[test_index[1]],task.test.labels[test_index[1]]))
    acc_test_d3.append(model.evaluate(task.test.images[test_index[2]],task.test.labels[test_index[2]]))

def save_acc(name,d):
	import json
	for i in range(3):
		path = './logs/permuted/acc{}/{}.txt'.format(str(i+1),name)
		with open(path,'w') as f:
			json.dump(d[i],f)

In [ ]:
def train(name):
    
    acc_test_d1 = []
    acc_test_d2 = []
    acc_test_d3 = []
    t1_x,t1_y = task.train.images[train_index[0]],task.train.labels[train_index[0]]
    t2_x,t2_y = task.train.images[train_index[1]],task.train.labels[train_index[1]]
    t3_x,t3_y = task.train.images[train_index[2]],task.train.labels[train_index[2]]
    
    #t4_x,t4_y = task[3].train.images,task[3].train.labels
    #t5_x,t5_y = task[4].train.images,task[4].train.labels
    
    
    
    model = cnn_model()
    model.val_data(task.validation.images[vali_index[0]],task.validation.labels[vali_index[0]])
    model.fit(t1_x,t1_y)
        
        

    test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3)
    
    if name == 'kal':
        model.transfer(t2_x,t2_y)
        test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3)
        model.transfer(t2_x,t2_y,num=2)
        test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3)
        model.transfer(t3_x,t3_y)
        test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3)
        model.transfer(t3_x,t3_y,num=3)
        test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3)
        '''
        model.transfer(t4_x,t4_y)
        test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3)
        model.transfer(t5_x,t5_y)
        test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3)
        '''
    if name == 'kal_pre':
        model.use_pre(t2_x,t2_y)
        test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3)
        model.use_pre(t3_x,t3_y)
        test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3)
    if name == 'kal_cur':
        model.use_cur(t2_x,t2_y)
        test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3)
        model.use_cur(t3_x,t3_y)
        test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3)
    if name == 'nor':
        model.fit(t2_x,t2_y)
        test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3)
        model.fit(t3_x,t3_y)
        test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3)
        '''
        model.fit(t4_x,t4_y)
        test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3)
        model.fit(t5_x,t5_y)
        test_acc(model,acc_test_d1,acc_test_d2,acc_test_d3)
        '''


    model.save(name)
    model.plot('res',name)
    
    return acc_test_d1,acc_test_d2,acc_test_d3

In [ ]:
task = Cifar10(True)
train_index,test_index,vali_index = create_disjoint_cifar10_task()